In [1]:
import csv
import numpy as np

"""Data loading"""
def load_results(path_dataset):
    """load data features."""
    to_int = dict(s = 1,b = 0)
    def convert(s):
        return to_int.get(s.decode("utf-8") , 0)
    
    data = data = np.genfromtxt(path_dataset, delimiter=",", skip_header=1, usecols=[1],
                                converters={1: convert})
    
    return data

def load_data_features(path_dataset):
    """load data features."""
    data = data = np.genfromtxt(path_dataset, delimiter=",", skip_header=1, 
                                usecols=tuple(range(2,32)))
    
    ids = np.genfromtxt(path_dataset, delimiter=",", skip_header=1, usecols=[0])
    
    return data, ids

In [3]:
def build_polynomial(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    Extended = np.empty((x.shape[0],0))
    
    for j in range(0, degree+1):
        for i in range(x.shape[1]):
            Extended = np.c_[Extended, x[:,i]**j]
          
    
    return Extended

In [4]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [5]:
def correlation_filter(X, y, threshold = 0.01):
    """Removes features which are correlated with y with less than threshold"""
    abs_corr = np.zeros(X.shape[1])
    for index, x in enumerate(X.T):
        abs_corr[index] = np.abs(np.corrcoef(y,x.T)[0,1])
        
    quality = np.where(abs_corr > threshold)
    
    return X[:,quality[0]], quality[0]

In [6]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss = calculate_loss(y, tx, w) + lambda_ * np.squeeze(w.T.dot(w))
    gradient = calculate_gradient(y, tx, w) + 2 * lambda_ * w
    # ***************************************************
    w -= gradient*gamma
    return loss, w

In [7]:
def normalize_data(data, mean = None, sigma = None):
    """Standardizes the data"""
    if mean is None:
        mean = np.nanmean(data[data != -999], axis = 0)
    
    if sigma is None:
        sigma = np.nanstd(data[data != -999], axis = 0)
    
    output = (data - mean)/sigma
    
    return output, mean, sigma

def standardize_data(data, min_value = None, max_value = None):
    """maps data to [0,1] range"""
    if min_value is None:
        min_value = np.min(data, axis = 0)
    
    if max_value is None:
        max_value = np.max(data, axis = 0)
        
    output = (data - min_value)/(max_value - min_value)
    
    return output, min_value, max_value

In [8]:
def create_subsets(data, y):
    """Creates four subsets based on the number of jets,
    which is 0, 1 and 2 or 3. 2 and 3 are put in one group,
    since they keep same features and have similar correlation patterns
    """
    data_subsets = []
    y_subsets=[]
    for i in range(3):
        if i ==2:
            mask = data[:,22] >= i
        else:
            mask = data[:,22] == i
        data_subsets.append(data[mask])
        y_subsets.append(y[mask])
        
    return data_subsets, y_subsets

In [9]:
def remove_zero_variance(data, mask = None):
    """removes zero variance columns based on the subset"""
    if mask is None:
        variance = np.var(data, axis = 0)
        mask = np.squeeze(~np.logical_or([variance ==0],[np.isnan(variance)]))
        
    return data[:, mask[:]], mask

In [10]:
def replace_missing(data, median = None):        
    """replaces nan by median value"""
    if median is None:
        median =[]
        for j in range(data.shape[1]):
            mask = data[:,j] != -999
            replace = np.median(data[mask,j])
            data[~mask,j] = replace
            median.append(replace)
    else:
        for j in range(data.shape[1]):
            mask = data[:,j] != -999
            data[~mask,j] = median[j]

    return data, median

In [26]:
def process_data(X_train, X_test, y_train, y_test):
    """Processes the test and training data by:
    -splitting data with respect to jet number, creating three groups
    -removing zero variance in each subgroup
    -removing columns which are lowly correlated to y
    -normalizing the data with mean and standard devation 
    -replacing -999 by median value of column"""
      
    train_subsets, y_train = create_subsets(X_train, y_train)
    test_subsets, y_test = create_subsets(X_test, y_test)
    
    for i in range(3):
        # change training sets
        train_subsets[i], mean, sigma =  normalize_data(train_subsets[i], mean = None, sigma = None)
        train_subsets[i], median = replace_missing(train_subsets[i], median = None)
        train_subsets[i], mask = remove_zero_variance(train_subsets[i])
        train_subsets[i], quality = correlation_filter(train_subsets[i], y_train[i], threshold = 0.01)
        
        #change test sets accordingly to training sets
        test_subsets[i], _, _ =  normalize_data(test_subsets[i], mean = mean, sigma = sigma)
        test_subsets[i], _ = replace_missing(test_subsets[i], median)
        test_subsets[i], _ = remove_zero_variance(test_subsets[i], mask)
        test_subsets[i] = test_subsets[i][:, quality]
        
    return train_subsets, test_subsets, y_train, y_test
        


In [12]:
def add_cross_terms(data):
    """Adds cross terms between columns"""
    enriched_data = data
    for x1 in data.T:
        for x2 in data.T:
            if np.sum(x1 - x2) != 0:
                enriched_data = np.c_[enriched_data, x1*x2]
                
    return enriched_data      

In [13]:
def add_log_terms(data):
    """Adds log terms to data"""
    extended = data
    for column in data.T:
        if np.sum(column <= -1) == 0:
            extended = np.c_[extended, np.log(1+ column)]
        
    return extended

In [14]:
def add_features(data, degree = None, sqrt = True, log = True, cross_terms = True):
    """Adds following features to data set:
    -log of features by log(1+x)
    -sqrt of features
    -polynomial extension of 0 up to degree
    -cross terms of features
    """ 
    #log
    if log:
        data = add_log_terms(data)
        output = data
    else:
        output = np.empty((data.shape[0],0))
        
    #polynomial
    if degree is not None:
        output = np.c_[output, build_polynomial(data, degree)]
      
    # add sqrt
    if sqrt:
        output = np.c_[output, np.sqrt(np.abs(data))]
        
    
    if cross_terms:
        output = np.c_[output, add_cross_terms(data)]
            
    return output

In [15]:
def compute_mse(y, tx, w):
    """compute the loss by mse."""
    e = y - tx.dot(w)
    mse = e.dot(e) / (2 * len(e))
    return mse


In [85]:
def stitch_solution(X_test, y_result):
    """Puts found y values back in right order for the complete data matrix,
    since it was split in four groups.
    X_test: original, preprocessed test data
    y_result: output of created model, list of three vectors containing predictions for group 1,2 and 3"""
    y_final=[]
    index_1 = 0
    index_2 = 0
    index_3 = 0
    for i in range(X_test.shape[0]):
        if X_test[i,22] == 0:
            y_final.append(y_result[0][index_1])
            index_1 += 1
            
        elif X_test[i,22] == 1:
            y_final.append(y_result[0][index_2])
            index_2 += 1
        
        elif X_test[i,22] > 1:
            y_final.append(y_result[0][index_3])
            index_3 += 1
         
    return y_final

In [33]:
#%% load data and process data
path =  "data"
X, _ = load_data_features(path +"/train.csv")
y = load_results(path +"/train.csv")
X_train, y_train = X[:int(0.8*len(X)),:], y[:int(0.8*len(X))]
X_test, y_test = X[int(0.8*len(X)):,:], y[int(0.8*len(X)):]
X_train, X_test_pro, y_train, y_test_pro = process_data(X_train, X_test, y_train, y_test)


C:\Users\jurri\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys


In [90]:
y_final = stitch_solution(X_test, y_result)
print(len(y_final))

50000


In [50]:
best_parameter_per_set = find_parameters(X_train, y_train, X_test_pro, y_test_pro)
y_result = []
for i in range(3):
    lambda_ = best_parameter_per_set[i][0,0]
    degree = int(best_parameter_per_set[i][0,1])
    X_train_ex = add_features(X_train[i], degree = degree, sqrt = True, log = True, cross_terms = True)
    X_test_pro_ex = add_features(X_test_pro[i], degree = degree, sqrt = True, log = True, cross_terms = True)
    
    show_result_ridge(X_train_ex, y_train[i], X_test_pro_ex, y_test_pro[i], lambda_)
    
    w = ridge_regression(y_train[i], X_train_ex, lambda_)
    y_result.append(X_test_pro_ex @ w)

y_final = stitch_solution(X_test, y_result)

Testing for jet 0
(1, 2) [[0. 0.]]
Start ridge regression test for degree 1 ...
Accuracy of the predictions is: 0.8414297889406929  and F-score is: 0.6539008644271802 with lambda: 1e-07
Precision is: 0.7410714285714286 and recall is: 0.5850793029175642
Start ridge regression test for degree 2 ...
Accuracy of the predictions is: 0.8420815160174462  and F-score is: 0.6558881363338431 with lambda: 1.5e-07
Precision is: 0.7417840375586855 and recall is: 0.587820638339534
(2,) [0 0]
Testing for jet 1
(1, 2) [[0. 0.]]
Start ridge regression test for degree 1 ...
Accuracy of the predictions is: 0.7881767026712373  and F-score is: 0.6928055529500047 with lambda: 1e-07
Precision is: 0.7156976744186047 and recall is: 0.6713324850027268
Start ridge regression test for degree 2 ...
Accuracy of the predictions is: 0.7899877110148115  and F-score is: 0.6951459956811566 with lambda: 1e-07
Precision is: 0.7188349514563107 and recall is: 0.6729685511725141
(2,) [1 1]
Testing for jet 2
(1, 2) [[0. 0.]]


NameError: name 'ramge' is not defined

In [48]:
def find_parameters(X_train, y_train, X_test, y_test):
    best_parameter_per_set = []
    losses_sets =[]
    degrees = np.linspace(1,22,21, dtype = int)
    lambdas = np.logspace(-9,2,22)
    degrees = [1, 2]
    lambdas = [0.0000001, 0.00000015]
    for i in range(3):
        print("Testing for jet",i)
        parameters, losses = hyper_optimizing(X_train[i], y_train[i], X_test[i], y_test[i],
                                methods = ["Ridge_regression"],
                                 lambdas = lambdas, degrees = degrees)
        
        best_parameter_per_set.append(parameters)
        losses_sets.append(losses)
    
    return best_parameter_per_set

In [49]:
def hyper_optimizing(X_train, y_train, X_test, y_test, methods = ["Ridge_regression"],
                     lambdas = [0.1], degrees = [1], gamma = 0.000001,  max_iter = 1):
    """Finds best lambda and degree to use on the given data, test possibilities are:
    -Ridge regression
    -Penalized Logistic regression"""
    
    # Check method names are correct
    if len([i for i in methods if i in ["Ridge_regression", "Penalized_logistic"]]) < len(methods):
        raise NameError("At least one method is wrong")
        
    #seed = 1
    #k_fold = 10
    #k_indices = build_k_indices(y_train, k_fold, seed)
    all_losses = np.zeros((len(methods), len(degrees), len(lambdas)))
    best_parameters = np.zeros((len(methods), 2))
    print(best_parameters.shape,best_parameters)
    for degree_index, degree in enumerate(degrees):
        X_train_ex = add_features(X_train, degree = degree, sqrt = True, log = True, cross_terms = True)
    
        for method_index, method in enumerate(methods):
            
            if method == "Ridge_regression":
                seed = 1
                k_fold = 10
                k_indices = build_k_indices(y_train, k_fold, seed)
                print("Start ridge regression test for degree", str(degree),"...")
                for index, lambda_ in enumerate(lambdas):
                    losses_te = []
                    for k in range(k_fold):
                        loss_te = cross_validation_ridge(y_train, X_train_ex, k_indices, k, lambda_)
                        losses_te.append(loss_te)
                    all_losses[method_index, degree_index, index] = np.mean(losses_te)
                 
                # Show percantage of correct results for this degree
                min_lambda = lambdas[np.argmin(all_losses[method_index, degree_index,:])]
                X_test_ex = add_features(X_test, degree = degree, sqrt = True, log = True, cross_terms = True)
                show_result_ridge(X_train_ex, y_train, X_test_ex, y_test, min_lambda)
                    
            elif method == "Penalized_logistic":
                #less k-fold for reason of speed
                seed = 1
                k_fold = 1
                k_indices = build_k_indices(y_train, k_fold, seed)
                print("Start penalized_logistic test...")
                for index, lambda_ in enumerate(lambdas):
                    losses_te = []
                    for k in range(k_fold):
                        loss_te, _ = cross_validation_logistic(y_train, X_train_ex, k_indices,
                                                    k, lambda_, gamma = 0.000001, max_iter = max_iter)
                        losses_te.append(loss_te)
                    all_losses[method_index, degree_index, index] = np.mean(losses_te) 
                
                # Show percantage of correct results for this degree
                min_lambda = lambdas[np.argmin(all_losses[method_index, degree_index,:])]
                X_test_ex = add_features(X_test, degree = degree, sqrt = True, log = True, cross_terms = True)
                show_result_logistic(X_train_ex, y_train, X_test_ex, y_test, min_lambda, gamma = 0.000001)
            
            
    print(all_losses)
    for k  in range(len(methods)):
        min_loss = np.argmin(all_losses[k,:,:], axis = 0)
        print(min_loss.shape, min_loss)
        best_parameters[k,0] = lambdas[min_loss[0]]
        best_parameters[k,1] = degrees[min_loss[1]]
        
    return best_parameters, all_losses
    

In [19]:
def cross_validation_ridge(y, x, k_indices, k, lambda_):
    """return the loss of ridge regression."""
    # ***************************************************
    y_test = y[k_indices[k]]
    x_test = x[k_indices[k], :]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_train = y[tr_indice]
    x_train = x[tr_indice, :]
    # ridge regression: 
    w = ridge_regression(y_train, x_train, lambda_)
    # ***************************************************
    # calculate the loss for train and test data:
    loss_te = np.sqrt(2*compute_mse(y_test, x_test, w))
    
    return loss_te

In [46]:
def show_result_ridge(X_train, y_train, X_test, y_test, lambda_):
    w = ridge_regression(y_train, X_train, lambda_)
    y_new = X_test @ w
    y_new[y_new<0.5]=0
    y_new[y_new>=0.5]=1
    summ = y_new + y_test
    TP = sum(summ == 2)
    TN = sum(summ == 0)
    diff = y_new - y_test
    FP = sum(diff == 1)
    FN = sum(diff == -1)
    accuracy = (TP+TN)/(TP +TN +FP + FN)
    F_score = TP/(TP + 0.5 * (FP +FN))
    recall = TP/(TP + FN)
    precision = TP/(TP + FP)
    print("Accuracy of the predictions is:",
          str(accuracy), " and F-score is:", str(F_score), "with lambda:",lambda_)
    print("Precision is:",str(precision), "and recall is:",str(recall))


In [21]:
def show_result_logistic(X_train, y_train, X_test, y_test, lambda_, gamma = 0.000001):
    w = np.zeros((x.shape[1], 1))
    _, w = learning_by_penalized_gradient(y_train, X_train, w, gamma, lambda_)
    y_new = X_test @ w
    # write function for this
    y_new[y_new<0.5]=0
    y_new[y_new>=0.5]=1
    summ = y_new + y_test
    TP = sum(summ == 2)
    TN = sum(summ == 0)
    diff = y_new - y_test
    FP = sum(diff == 1)
    FN = sum(diff == -1)
    accuracy = (TP+TN)/(TP +TN +FP + FN)
    F_score = TP/(TP + 0.5 * (FP +FN))
    print("Accuracy of the predictions is:",
          str(accuracy), " and F-score is:", str(F_score), "with lambda:",lambda_)


In [22]:
def cross_validation_logistic(y, x, k_indices, k,lambda_, gamma,max_iter):
    """return the loss of ridge regression."""
    # split according to k_indices
    y_test = y[k_indices[k]]
    x_test = x[k_indices[k], :]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_train = y[tr_indice]
    x_train = x[tr_indice, :]
    
    w = np.zeros((x.shape[1], 1))
    threshold = 1e-8
    losses = []
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y_train, x_train, w, gamma, lambda_)
        # log info
        if iter % 999 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # check loss actually decreases, if not decrease gamma
        if iter > 0:
            if loss > losses[-1]:
                gamma = gamma/2
            
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
        
    # calculate the loss for train and test data:
    loss_te = calculate_loss(y_test, x_test, w)
    
    return loss_te, w

In [23]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    # ***************************************************
    if len(tx.shape) > 1:
        w = np.linalg.solve(tx.T @ tx + (2*tx.shape[0]*lambda_)*np.identity(tx.shape[1]), tx.T @ y)
    else:
        w = 1/(tx.T @ tx + lambda_) * tx.T @ y                        
    # ***************************************************
    return w



In [24]:
def sigmoid(t):
    """apply the sigmoid function on t."""
    return 1/(1+np.exp(-t))

def calculate_loss(y, tx, w):
    """compute the loss: negative log likelihood."""
    inter_y = y.reshape(len(y),1)
    z = tx @ w
    a = np.sum(np.log(1 + np.exp(z)))
    b = inter_y.T @ z
    loss = a - b
    return np.squeeze(loss)

def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    inter_y = y.reshape(len(y),1)
    gradient = tx.T @ (sigmoid(tx @ w) - inter_y)
    return gradient

In [247]:
w = np.zeros((X_train[1].shape[1], 1))
threshold = 1e-8
losses = []
max_iter = 10000
lambda_ = 0
gamma = 0.000001
#start the logistic regression
for iter in range(max_iter):
     # get loss and update w.
    loss, w = learning_by_penalized_gradient(y_train[1], X_train[1], w, gamma, lambda_)
        # log info
    if iter % 100 == 0:
        print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
    # converge criterion
    losses.append(loss)
    if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
        break

y_new = X_test[1] @ w
y_new[y_new<0.5]=0
y_new[y_new>=0.5]=1
correct = y_new.T - y_test[1]
correct[correct!=0]=1
print(1 - np.sum(correct)/len(correct[0,:]))

Current iteration=0, loss=43032.65641070308
Current iteration=100, loss=38034.65434628601
Current iteration=200, loss=37639.27664896377
Current iteration=300, loss=37418.889681703076
Current iteration=400, loss=37274.741199240416
Current iteration=500, loss=37176.59226694897
Current iteration=600, loss=37107.69305313046
Current iteration=700, loss=37057.97530938601
Current iteration=800, loss=37021.18935277108
Current iteration=900, loss=36993.352056442345
Current iteration=1000, loss=36971.86109215789
Current iteration=1100, loss=36954.97566067417
Current iteration=1200, loss=36941.504774259214
Current iteration=1300, loss=36930.61588624264
Current iteration=1400, loss=36921.71490249629
Current iteration=1500, loss=36914.36945423421
Current iteration=1600, loss=36908.25892183308
Current iteration=1700, loss=36903.14131117387
Current iteration=1800, loss=36898.83093163525
Current iteration=1900, loss=36895.183110014565
Current iteration=2000, loss=36892.083557627666
Current iteration=2

In [305]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]
            
def stochastic_gradient_descent(
        y, tx, initial_w, batch_size, max_iters, gamma, lambda_):
    """Stochastic gradient descent algorithm."""
    # ***************************************************
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss = calculate_loss(y, tx, w) + lambda_ * np.squeeze(w.T.dot(w))
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gradient = calculate_gradient(y, tx, w) + 2 * lambda_ * w
            # ***************************************************
            # update w by gradient
            w -= gamma * gradient   
            
        # store w and loss
        ws.append(w)
        losses.append(loss)
        if n_iter % 100 == 0:
            print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
                  bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
            
    # ***************************************************
    return losses, ws

In [138]:
w = np.zeros((X_test[1].shape[1], 1))
threshold = 1e-8
losses = []
max_iter = 3000
lambda_ = 0.08
gamma = 0.00001
batch_size = 5000
max_iters = 1000

losses, ws = stochastic_gradient_descent(y_train[1], X_train[1], w, batch_size, max_iter, gamma, lambda_)

Gradient Descent(0/2999): loss=43032.65641070308, w0=[-0.00015543], w1=[0.00057892]
Gradient Descent(100/2999): loss=38308.48336908368, w0=[-0.01301644], w1=[0.0051777]
Gradient Descent(200/2999): loss=38063.16392576093, w0=[-0.02341128], w1=[0.00713485]
Gradient Descent(300/2999): loss=37967.2307168726, w0=[-0.03316536], w1=[0.00854527]
Gradient Descent(400/2999): loss=37894.802331778694, w0=[-0.04267698], w1=[0.00971019]
Gradient Descent(500/2999): loss=37831.9663784284, w0=[-0.05205284], w1=[0.0107498]
Gradient Descent(600/2999): loss=37775.452820463935, w0=[-0.06132667], w1=[0.01172069]
Gradient Descent(700/2999): loss=37723.94678832, w0=[-0.07051208], w1=[0.01265249]
Gradient Descent(800/2999): loss=37676.72611830914, w0=[-0.0796165], w1=[0.01356169]
Gradient Descent(900/2999): loss=37633.29941362311, w0=[-0.08864521], w1=[0.01445768]
Gradient Descent(1000/2999): loss=37593.28328453124, w0=[-0.09760252], w1=[0.01534588]
Gradient Descent(1100/2999): loss=37556.35561975944, w0=[-0.1

In [51]:
a= y.reshape(250000,1)

In [105]:
1/(1+np.exp(-227.91778431))

1.0

In [68]:
np.var(X, axis = 0)

array([1.65116124e+05, 1.24925594e+03, 1.66697530e+03, 4.05202959e+03,
       2.06551758e+05, 4.32925819e+05, 2.05226188e+05, 6.12947368e-01,
       4.96106539e+02, 1.33878515e+04, 7.13587788e-01, 1.42463906e+00,
       2.05749162e+05, 5.02299351e+02, 1.47398106e+00, 3.30061476e+00,
       4.86858853e+02, 1.60017344e+00, 3.30006328e+00, 1.08205651e+03,
       3.28413798e+00, 1.60020609e+04, 9.55358361e-01, 2.84048199e+05,
       2.39451000e+05, 2.39446692e+05, 2.30279570e+05, 2.05556795e+05,
       2.05560779e+05, 9.60703157e+03])